# Continuous Integration/Continuous Deployment (CI/CD)

## Continuous Integration (CI)
Change management is essential in software development. Once you make a change, you need to merge your changes where other people in your team can see it and ultimately use it. One of the most prominent version control applications in github.

Each time you integrate your code with the source, an automatic *build* process should be triggered that compiles the code with new changes and possibly runs tests.

The main goal in this step is to catch and address the integration errors as early as possible in the development lifecycle, reducing the risk of conflicts and ensuring that the codebase remains stable.

## Continuous Deployment (CD)

After the build, we need to make the executables available for users in a target environment as well as adjust server configurations, transfer any files and update the databases if applicable. 



